In [1]:
import pandas as pd
import numpy as np
import preprocess_helper
import os
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import enchant
checker = enchant.Dict("en_US")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\11099\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Use the processed ASRS output from `load_asrs()` in `preprocess_helper.py`.

In [2]:
# Dictionaries of acronym
# CASA = pd.read_csv('dictionaries/CASA.csv')
FAA = pd.read_csv('dictionaries/FAA.csv')
NASA = pd.read_csv('dictionaries/nasa_abbr.csv')
top_thirty = pd.read_csv('dictionaries/FAA Core Airports 2014.csv')

# Limit to top thirty airports
ASRS = preprocess_helper.load_asrs(load_saved=True)
top_thirty_bool = ASRS['tracon_code'].isin(top_thirty['tracon_key'])
ASRS_30 = ASRS[top_thirty_bool]

# Accident and incident counts of FAA and NTSB
acc_inc = pd.read_csv(r"E:\aviation_integrated\results\combined_vol_incident.csv")

D:\Anaconda\envs\aviation_integrated\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8,9,11,12,14,15,19,20,28,29,30,31,32,33,34,38,39,40,41,42,46,50,53,54,56,57,58,59,60,64,65,70,71,72,74,75,80,81,82,86,88,93,94,95,104) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


`narrative` is the combination of `narrative_report1` and `narrative_report2`.

In [3]:
# Utility function used for counting the acronyms in ASRS data

# Check if a word is spelled correctly 
def spell_check(word):
    return checker.check(word)

# Count the total occurrences of all dictionary acronyms in the text 
def count_total(word_count, dictionary):
    total = 0
    
    for word in dictionary:
        #if (word_count[word] != 0): print(word)
        total += word_count[word]
        
    return total

# Count the total unique occurrences of all dictionary acronyms in the text 
def count_unique(word_count, dictionary):
    total = 0
    
    for word in dictionary:
        if (word_count[word] != 0):
            total += 1
        
    return total

# Remove target symbols at both the start and the end of each string 
def strip_symbol(list_words, symbols):
    return list(map(lambda x: x.strip(symbols), list_words))

Note: It seems Jimmy had already removed all symbols in the `clean_string_and_tokenize` function.

**Update in 01/04/2022: Discard CASA dictionary temporarily and stop testing spelling check.** 

In [4]:
# Clean the field of acronyms

# CASA['acronym'] = CASA['acronym'].str.strip().str.lower()
# CASA['acronym'] = CASA['acronym'].str.replace(r'\(.+\)', '', regex=True)
# CASA.drop_duplicates(["acronym"], inplace=True, keep='first')
# CASA_cleaned = CASA[['acronym']]

FAA['acronym'] = FAA['acronym'].str.strip().str.lower()
FAA.drop_duplicates(["acronym"], inplace=True, keep='first')
FAA_cleaned = FAA[['acronym']]

NASA['acronym'] = NASA['acronym'].str.strip().str.lower()
NASA.drop_duplicates(["acronym"], inplace=True, keep='first')
NASA_cleaned = NASA[['acronym']]

top_thirty['tracon_key'] = top_thirty['tracon_key'].str.strip().str.upper()
top_thirty.drop_duplicates(["tracon_key"], inplace=True, keep='first')
top_thirty_cleaned = top_thirty[['tracon_key']]

# Remove stop words in the dictionaries
stop_words = set(stopwords.words('english'))

# bool_index = CASA_cleaned.isin(stop_words)['acronym'].values
# CASA_cleaned = CASA_cleaned[~bool_index]

bool_index = FAA_cleaned.isin(stop_words)['acronym'].values
FAA_cleaned = FAA_cleaned[~bool_index]

bool_index = NASA_cleaned.isin(stop_words)['acronym'].values
NASA_cleaned = NASA_cleaned[~bool_index]

# Filter out the words passing the spelling check (OBSOLETE)
# bool_index = np.array(list(map(spell_check, list(CASA_cleaned['acronym'].values))))
# CASA_cleaned = CASA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(FAA_cleaned['acronym'].values))))
# FAA_cleaned = FAA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(NASA_cleaned['acronym'].values))))
# NASA_cleaned = NASA_cleaned[~bool_index]

**Update in 01/07/2022: Use the top 30 airports listed in `FAA Core Airports 2014.csv`**

****
## Create Master Key

In [5]:
date = pd.date_range(start = '1/1/1988', end = '12/31/2019', freq = 'M')
date_df = pd.DataFrame(data = {'year': date.year, 'month': date.month})
master_key = top_thirty.merge(date_df, how = 'cross')
master_key = master_key.drop('ATADS_Type', axis = 1)
master_key = master_key.rename({'tracon_key': 'tracon'}, axis = 1)
master_key

,tracon,year,month
0,ATL,1988,1
1,ATL,1988,2
2,ATL,1988,3
3,ATL,1988,4
4,ATL,1988,5
...,...,...,...
11515,TPA,2019,8
11516,TPA,2019,9
11517,TPA,2019,10
11518,TPA,2019,11


In [6]:
master_key.to_csv("quality_check/master_key.csv")

## Merge FAA/NTSB with master key

In [7]:
# Limit NTSB/FAA to top 30
acc_inc_bool = acc_inc['airport_code'].isin(top_thirty['tracon_key'])
acc_inc_30 = acc_inc.loc[acc_inc_bool]
acc_inc_30 = acc_inc_30.drop("Unnamed: 0", axis = 1)
acc_inc_30 = acc_inc_30.rename({'airport_code': 'tracon'}, axis = 1)
acc_inc_30.to_csv('quality_check/acc_inc_30.csv')

In [8]:
faa_ntsb_merge = master_key.merge(acc_inc_30, left_on=['tracon', 'year', 'month'], right_on=['tracon', 'year', 'month'], how='left')
faa_ntsb_merge.to_csv('quality_check/merge_acc_inc.csv')
faa_ntsb_merge

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups
0,ATL,1988,1,0.0,0.0,1.0,0.0,1.0,1.0
1,ATL,1988,2,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,1988,3,1.0,0.0,4.0,1.0,5.0,4.0
3,ATL,1988,4,1.0,0.0,0.0,0.0,1.0,1.0
4,ATL,1988,5,0.0,2.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
11515,TPA,2019,8,0.0,1.0,0.0,0.0,1.0,1.0
11516,TPA,2019,9,0.0,0.0,2.0,0.0,2.0,2.0
11517,TPA,2019,10,NaN,NaN,NaN,NaN,NaN,NaN
11518,TPA,2019,11,NaN,NaN,NaN,NaN,NaN,NaN


Note: Perhaps mark the missing data with an indicator column?

In [9]:
faa_ntsb_merge = faa_ntsb_merge.fillna(0)

## Moving average of ASRS

In [42]:
# collapse ASRS by [year, month, tracon_code]
asrs_collapse = ASRS_30.groupby(['year', 'month', 'tracon_code'])[['narrative']].agg(lambda x: x.str.cat(sep=' ')).reset_index()
asrs_collapse['num_obs'] = ASRS_30.groupby(['year', 'month', 'tracon_code']).size().values
asrs_collapse['AtcAdvisoryMultCount'] = ASRS_30.groupby(['year', 'month', 'tracon_code'])['AtcAdvisoryMultCount'].sum().values

In [43]:
asrs_collapse.head()

,year,month,tracon_code,narrative,num_obs,AtcAdvisoryMultCount
0,1988.0,1.0,ATL,apching the atl area a solid line of thunderst...,3,3
1,1988.0,1.0,BOS,atis and arpt rwy condition broadcast was givi...,5,5
2,1988.0,1.0,CLT,we were on a radar vector by clt apch ctl on a...,1,1
3,1988.0,1.0,DCA,i departed bwi vfr and dep ctl clred me on cou...,5,6
4,1988.0,1.0,DEN,a vfr flt being conducted under far part 91 wa...,7,8


In [50]:
# merge ASRS with master key
asrs_merge = master_key.merge(asrs_collapse, left_on=['tracon', 'year', 'month'], right_on=['tracon_code', 'year', 'month'], how='left')
asrs_merge = asrs_merge.loc[:, ['year', 'month', 'tracon', 'narrative', 'AtcAdvisoryMultCount', 'num_obs']]
asrs_merge['narrative'] = asrs_merge['narrative'].fillna('')
asrs_merge = asrs_merge.fillna(0)
asrs_merge.head(10)

,year,month,tracon,narrative,AtcAdvisoryMultCount,num_obs
0,1988,1,ATL,apching the atl area a solid line of thunderst...,3.0,3.0
1,1988,2,ATL,established on ils rwy 27l atl on a 12 mile fi...,5.0,5.0
2,1988,3,ATL,f o was flying the acft and assigned a 060 deg...,7.0,7.0
3,1988,4,ATL,tca was inadvertently entered at 11500 in uppe...,1.0,1.0
4,1988,5,ATL,,0.0,0.0
5,1988,6,ATL,while being vectored for apch to rwy 9r atc di...,5.0,5.0
6,1988,7,ATL,during enrte portion of flt we were assigned a...,5.0,3.0
7,1988,8,ATL,at about 200 agl on final apch to rwy 27l at a...,11.0,8.0
8,1988,9,ATL,mlg x was dsnded to 12000 mlg x read back 1000...,6.0,5.0
9,1988,10,ATL,at 13000 level flt 250 kts i asked the f o to ...,5.0,4.0


In [11]:
# create moving average window 
window_len = 6
lag = 2
windows = asrs_merge.rolling(window = window_len, min_periods=window_len)

In [53]:
window_len = 6
lag = 1

asrs_splits = [asrs_merge[asrs_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
faa_ntsb_splits = [faa_ntsb_merge[faa_ntsb_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
final = []

for i in range(len(asrs_splits)):
    asrs = asrs_splits[i].reset_index().drop('index', axis = 1)
    faa_ntsb = faa_ntsb_splits[i].reset_index().drop('index', axis = 1)
    
    # compute moving average of ASRS 
    windows = asrs.rolling(window = window_len, min_periods=window_len)
    num_obs_sum = []
    num_obs_avg = []
    num_word_sum = []
    num_word_avg = []
    num_unique_word_sum = []
    num_unique_word_avg = []
    FAA_total_sum = []
    FAA_total_avg = []
    FAA_unique_sum = []
    FAA_unique_avg = []
    NASA_total_sum = []
    NASA_total_avg = []
    NASA_unique_sum = []
    NASA_unique_avg = []
    AtcAdvisoryMultCount_sum = []
    AtcAdvisoryMultCount_avg = []


    for window in windows:
        size = window.shape[0]

        # create a dictionary of counts of words
        all_words = window['narrative'].str.cat(sep=' ')
        all_words = all_words.lower().split()
        all_words = [x.strip('{[(,:.') for x in all_words]
        words_count = Counter(all_words)
        
        # number of observations
        num_obs = window['num_obs'].sum()
        total_obs = num_obs * size
        num_obs_sum.append(num_obs)
        num_obs_avg.append(num_obs / size)
        
        # number of words
        num_word = len(all_words)
        num_word_sum.append(num_word)
        num_word_avg.append(num_word / total_obs)

        # number of unique words
        num_unique_word = len(words_count)
        num_unique_word_sum.append(num_unique_word)
        num_unique_word_avg.append(num_unique_word / total_obs)

        # number of FAA acronyms
        FAA_total = count_total(words_count, dictionary=FAA_cleaned['acronym'].values)
        FAA_total_sum.append(FAA_total)
        FAA_total_avg.append(FAA_total / total_obs)

        # number of unique FAA acronyms
        FAA_unique = count_unique(words_count, dictionary=FAA_cleaned['acronym'].values)
        FAA_unique_sum.append(FAA_unique)
        FAA_unique_avg.append(FAA_unique / total_obs)

        # number of NASA acronyms
        NASA_total = count_total(words_count, dictionary=NASA_cleaned['acronym'].values)
        NASA_total_sum.append(NASA_total)
        NASA_total_avg.append(NASA_total / total_obs)

        # number of unique NASA acronyms
        NASA_unique = count_unique(words_count, dictionary=NASA_cleaned['acronym'].values)
        NASA_unique_sum.append(NASA_unique)
        NASA_unique_avg.append(NASA_unique / total_obs)

        # number of atcadvisory splits
        num_splits = window['AtcAdvisoryMultCount'].sum()
        AtcAdvisoryMultCount_sum.append(num_splits)
        AtcAdvisoryMultCount_avg.append(num_splits / total_obs)

    num_word_sum = np.array(num_word_sum)
    num_word_avg =  np.array(num_word_avg)
    num_unique_word_sum = np.array(num_unique_word_sum)
    num_unique_word_avg = np.array(num_unique_word_avg)
    FAA_total_sum = np.array(FAA_total_sum)
    FAA_total_avg = np.array(FAA_total_avg)
    FAA_unique_sum = np.array(FAA_unique_sum)
    FAA_unique_avg = np.array(FAA_unique_avg)
    NASA_total_sum = np.array(NASA_total_sum)
    NASA_total_avg = np.array(NASA_total_avg)
    NASA_unique_sum = np.array(NASA_unique_sum)
    NASA_unique_avg = np.array(NASA_unique_avg)
    AtcAdvisoryMultCount_sum = np.array(AtcAdvisoryMultCount_sum)
    AtcAdvisoryMultCount_avg = np.array(AtcAdvisoryMultCount_avg)
    num_obs_sum = np.array(num_obs_sum)
    num_obs_avg = np.array(num_obs_avg)
    
    asrs_moving = asrs.drop(['narrative', 'AtcAdvisoryMultCount'], axis = 1)
    asrs_moving['num_obs_sum'] = num_obs_sum
    asrs_moving['num_obs_avg'] = num_obs_avg
    asrs_moving['num_word_sum'] = num_word_sum
    asrs_moving['num_word_avg'] = num_word_avg
    asrs_moving['num_unique_word_sum'] = num_unique_word_sum 
    asrs_moving['num_unique_word_avg'] = num_unique_word_avg 
    asrs_moving['FAA_total_sum'] = FAA_total_sum
    asrs_moving['FAA_total_avg'] = FAA_total_avg
    asrs_moving['FAA_unique_sum'] = FAA_unique_sum
    asrs_moving['FAA_unique_avg'] = FAA_unique_avg
    asrs_moving['NASA_total_sum'] = NASA_total_sum
    asrs_moving['NASA_total_avg'] = NASA_total_avg
    asrs_moving['NASA_unique_sum'] = NASA_unique_sum
    asrs_moving['NASA_unique_avg'] = NASA_unique_avg
    asrs_moving['AtcAdvisoryMultCount_sum'] = AtcAdvisoryMultCount_sum
    asrs_moving['AtcAdvisoryMultCount_avg'] = AtcAdvisoryMultCount_avg
    
    # insert empty rows at the beginning for lagging
    empty_row_df = pd.DataFrame(data = [pd.Series(index=asrs.columns) for _ in range(lag)])
    empty_row_df = empty_row_df.drop('tracon', axis = 1)   
    
    # merge the moving averages of ASRS with FAA+NTSB 
    asrs_moving = pd.concat([empty_row_df, asrs_moving])
    asrs_moving = asrs_moving.drop(['year', 'month', 'tracon'], axis = 1)
    asrs_moving = asrs_moving.reset_index().drop('index', axis = 1)
    merge_df = pd.concat([faa_ntsb, asrs_moving], axis = 1)
    final.append(merge_df)

final_df = pd.concat(final).reset_index().drop(['index', 'AtcAdvisoryMultCount', 'narrative'], axis = 1)
final_df.to_csv('quality_check/final.csv')

D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:118: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:50: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:55: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:60: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:65: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:70: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/bin/xpython:75: RuntimeWarning: invalid value encountered in double_scalars
D:/Anaconda/envs/aviation_integrated/Library/b

In [17]:
asrs_merge.to_csv("quality_check/asrs_merge.csv")
asrs_moving.to_csv("quality_check/asrs_moving.csv")

****
## Create moving average of ASRS

# Quality check

In [ ]:
try:
    os.mkdir("quality_check")
except OSError as error:
    print(error)

np.random.seed(8)   
final_merge.sample(10).to_csv("quality_check/final_merge_10.csv")